In [12]:
import csv

file = 'cleaned2.csv'

with open(file, 'r') as fi:
    reader = csv.reader(fi)
    headers = next(reader)
    for i in range(len(headers)):
        print(i, headers[i])     

0 HRHHID
1 HETENURE
2 HETELHHD
3 HEFAMINC
4 HRNUMHOU
5 GEREG
6 GEDIV
7 GTINDVPC
8 PRTAGE
9 PEMARITL
10 PESEX
11 PEAFEVER
12 PEEDUCA
13 PTDTRACE
14 PUCHINHH
15 PEHSPNON
16 PENATVTY
17 PRCITSHP
18 PUWK
19 PUBUS1
20 PUBUS2OT
21 PUDIS
22 PERET1
23 PRIOELG
24 PRCOW1
25 PRCOW2
26 PRCOWPG
27 PEERNPER
28 PEERNHRY
29 PEERNH2
30 PEERNHRO
31 PEERNLAB
32 PRCHLD
33 race-binary


In [9]:
with open(file, 'r') as fi:
    reader = csv.reader(fi)
    headers = next(reader)
    
    white = []
    nonwhite = []
    income = []
    
    for line in reader:
        race = line[35]
        
        # income 
        
        
        # remove race
        line.pop(13)
        if line[35] == 'white':
            white.append(line)
        else:
            nonwhite.append(line)